# Installs and Imports


In [4]:
import os

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import named_tuple_factory

import pandas as pd
import openai

from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores.cassandra import Cassandra
from langchain_openai import OpenAI
from langchain_openai import OpenAIEmbeddings


### Connect to DataStax

In [ ]:
# This secure connect bundle is autogenerated when you donwload your SCB, 
# if yours is different update the file name below
cloud_config= {'secure_connect_bundle': 'datastax_auth/secure-connect-fiddlerai.zip'}

ASTRA_DB_APPLICATION_TOKEN = os.environ.get('ASTRA_DB_APPLICATION_TOKEN')
print(ASTRA_DB_APPLICATION_TOKEN)

auth_provider=PlainTextAuthProvider("token", ASTRA_DB_APPLICATION_TOKEN)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)

session = cluster.connect()

session.set_keyspace('fiddlerai')

In [ ]:

llmProvider = 'openai'  # 'GCP_VertexAI', 'Azure_OpenAI'
table_name = 'fiddler_doc_snippets_' + llmProvider
os.environ["OPENAI_API_TYPE"] = "open_ai"


llm = OpenAI(temperature=0)

# Explicitly use the desired model and configure its dimension size
myEmbedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
    model_kwargs={"dimensions": 1536}
)
print(f"LLM from OpenAI, Embeddings from OpenAI using model '{myEmbedding.model}' with 1536 dimensions.")

In [ ]:
# Preprocessed data 
file_path = 'documentation_data/vector_index_feed_v25.10.csv'

df = pd.read_csv(file_path)
# df.filter(items=['row_id', 'vector', 'body_blob', 'metadata_s'])
df.info()

#### Please double-check your dataframe before running the next cell. 
##### Running the next cell will delete the existing snippets from the last version of the docs. 

In [ ]:
# Clear the database table prior to refill

session.execute(f"TRUNCATE TABLE {table_name}")

In [ ]:
# 1. Load the pre-chunked documents from the DataFrame
# Each row in the CSV is already a perfectly sized chunk from the preprocessing notebook.
print("Loading pre-chunked documents from the CSV...")
loader = DataFrameLoader(df, page_content_column="text")
documents = loader.load()
print(f"Loaded {len(documents)} document chunks.")

# 2. Initialize the vector store directly
# We don't need a text_splitter because the documents are already chunked.
print(f"Initializing Cassandra vector store with table '{table_name}'...")
vector_store = Cassandra(
    embedding=myEmbedding,
    session=session,
    keyspace="fiddlerai",
    table_name=table_name,
)

# 3. Add the documents to the vector store
# This will create embeddings for each document chunk and store them in Cassandra.
print(f"Adding {len(documents)} chunks to the vector store. This may take a few minutes...")
vector_store.add_documents(documents)

print("\n✅ Success! All document chunks have been embedded and stored in Cassandra.")

In [ ]:
# One-liner test
def quick_test():
    query = "What is Fiddler?"
    docs = vector_store.similarity_search(query, k=2)
    return f"Retrieved {len(docs)} docs. First doc preview: {docs[0].page_content[:100] if docs else 'None'}"

print(quick_test())

### You are done. Please check the Chatbot to ensure it is working and returning answers from the lastest docs we just uploaded.

## Sample code for querying the datastax Vector DB to ensure we have the right data there

In [ ]:
# UTILITY: Check table structure using system schema queries
print("=== TABLE INFORMATION ===")

# Temporarily set row factory to get normal Cassandra Row objects
original_row_factory = session.row_factory
session.row_factory = named_tuple_factory

try:
    # Get table info
    table_info_query = """
    SELECT table_name, bloom_filter_fp_chance, caching, comment, compaction, compression 
    FROM system_schema.tables 
    WHERE keyspace_name = 'fiddlerai' AND table_name = %s
    """

    rows = session.execute(table_info_query, [table_name])
    for row in rows:
        print(f"Table: {row.table_name}")
        print(f"Compaction: {row.compaction}")
        print(f"Compression: {row.compression}")
        print(f"Caching: {row.caching}")

    print("\n=== COLUMN INFORMATION ===")

    # Get column info  
    columns_query = """
    SELECT column_name, type, kind, position 
    FROM system_schema.columns 
    WHERE keyspace_name = 'fiddlerai' AND table_name = %s
    --ORDER BY position
    """

    rows = session.execute(columns_query, [table_name])
    for row in rows:
        print(f"Column: {row.column_name}, Type: {row.type}, Kind: {row.kind}")

    print("\n=== ROW COUNT (APPROXIMATE) ===")
    # Get approximate row count
    count_query = f"SELECT COUNT(*) FROM fiddlerai.{table_name}"
    try:
        result = session.execute(count_query)
        for row in result:
            print(f"Approximate row count: {row.count}")
    except Exception as e:
        print(f"Note: COUNT(*) may timeout on large tables. Error: {e}")
        
    print("\n=== SAMPLE DATA ===")
    # Show sample data
    sample_query = f"SELECT * FROM fiddlerai.{table_name} LIMIT 3"
    rows = session.execute(sample_query)
    for i, row in enumerate(rows):
        print(f"\nSample row {i+1}:")
        print(f"  Row ID: {getattr(row, 'row_id', 'N/A')}")
        if hasattr(row, 'body_blob'):
            print(f"  Content preview: {str(row.body_blob)[:100]}...")
        if hasattr(row, 'vector') and row.vector:
            print(f"  Vector dimensions: {len(row.vector)}")
        if hasattr(row, 'metadata_s'):
            print(f"  Metadata: {row.metadata_s}")

finally:
    # Restore the original pandas factory
    session.row_factory = original_row_factory
    print("\n=== Row factory restored ===")

In [ ]:
cqlSelect = f'SELECT * FROM fiddlerai.{table_name} LIMIT 10;'  # (Not a production-optimized query ...)

rows = session.execute(cqlSelect)
print(rows)
for row_i, row in enumerate(rows):
    print(f'\nRow {row_i}:')
    print(row)
    # depending on the cassIO version, the underlying Cassandra table can have different structure ...
    #try:
        # you are using the new cassIO 0.1.0+ : congratulations :)
       # print(f'    row_id:            {row.row_id}')
        #print(f'    vector:            {str(row.vector)[:64]} ...')
       # print(f'    body_blob:         {row.body_blob} ...')
       # print(f'    metadata_s:        {row.metadata_s}')        
    #except AttributeError:
        # Please upgrade your cassIO to the latest version ...
    print(f'    document_id:      {row.row_id}')
    print(f'    embedding_vector: {str(row.vector)[:64]} ...')
    print(f'    document:         {row.body_blob[:64]} ...')
    print(f'    metadata_blob:    {row.metadata_s}')

print('\n...')

In [ ]:
import pandas as pd

cql_query = f'SELECT * FROM fiddlerai.{table_name};'

try:
    print("Executing query and loading data into pandas DataFrame...")
    rows = session.execute(cql_query)

    # 2. Convert the Cassandra ResultSet to a pandas DataFrame
    df = pd.DataFrame(list(rows))

    if df.empty:
        print("Query returned no data. CSV file will not be created.")
    else:
        # 3. Save the DataFrame to a CSV file
        csv_file_path = f'{table_name}_output_pandas.csv'
        
        # The index=False argument prevents pandas from writing the DataFrame index as a column
        df.to_csv(csv_file_path, index=False, encoding='utf-8')
        
        print(f"✅ Success! Data has been saved to '{csv_file_path}'")

finally:
    if 'cluster' in locals():
        print('in locals')
    if cluster.is_shutdown:
        print('is shutdown')

In [ ]:
createTableSQL = """CREATE TABLE fiddler_chatbot_history (
    row_id text PRIMARY KEY,
    response text,
    response_vector vector<float, 1536>,
    source_docs text,
    source_docs_vector vector<float, 1536>,
    question text,
    question_vector vector<float, 1536>,
    comment text,
    feedback int,
    metadata_s map<text, text>,
    ts timestamp)"""
# ) WITH additional_write_policy = '99p'
#     AND bloom_filter_fp_chance = 0.01
#     AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
#     AND comment = ''
#     AND compaction = {'class': 'org.apache.cassandra.db.compaction.UnifiedCompactionStrategy'}
#     AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
#     AND crc_check_chance = 1.0
#     AND default_time_to_live = 0
#     AND gc_grace_seconds = 864000
#     AND max_index_interval = 2048
#     AND memtable_flush_period_in_ms = 0
#     AND min_index_interval = 128
#     AND read_repair = 'BLOCKING'
#     AND speculative_retry = '99p';"""

In [ ]:
session.execute(createTableSQL)

In [ ]:
EMBEDDING_MODEL = "text-embedding-3-large"
response = openai.Embedding.create(
    model=EMBEDDING_MODEL, input="How are you doing", dimensions=1536
)
response

In [ ]:
# CORRECTED: Check table structure using system schema queries
from cassandra.query import named_tuple_factory
print("=== TABLE INFORMATION ===")

# Temporarily set row factory to get normal Cassandra Row objects
original_row_factory = session.row_factory
session.row_factory = named_tuple_factory

try:
    # Get table info
    table_info_query = """
    SELECT table_name, bloom_filter_fp_chance, caching, comment, compaction, compression 
    FROM system_schema.tables 
    WHERE keyspace_name = 'fiddlerai' AND table_name = %s
    """

    rows = session.execute(table_info_query, [table_name])
    for row in rows:
        print(f"Table: {row.table_name}")
        print(f"Compaction: {row.compaction}")
        print(f"Compression: {row.compression}")
        print(f"Caching: {row.caching}")

    print("\n=== COLUMN INFORMATION ===")

    # Get column info  
    columns_query = """
    SELECT column_name, type, kind, position 
    FROM system_schema.columns 
    WHERE keyspace_name = 'fiddlerai' AND table_name = %s
    --ORDER BY position
    """

    rows = session.execute(columns_query, [table_name])
    for row in rows:
        print(f"Column: {row.column_name}, Type: {row.type}, Kind: {row.kind}")

    print("\n=== ROW COUNT (APPROXIMATE) ===")
    # Get approximate row count
    count_query = f"SELECT COUNT(*) FROM fiddlerai.{table_name}"
    try:
        result = session.execute(count_query)
        for row in result:
            print(f"Approximate row count: {row.count}")
    except Exception as e:
        print(f"Note: COUNT(*) may timeout on large tables. Error: {e}")
        
    print("\n=== SAMPLE DATA ===")
    # Show sample data
    sample_query = f"SELECT * FROM fiddlerai.{table_name} LIMIT 3"
    rows = session.execute(sample_query)
    for i, row in enumerate(rows):
        print(f"\nSample row {i+1}:")
        print(f"  Row ID: {getattr(row, 'row_id', 'N/A')}")
        if hasattr(row, 'body_blob'):
            print(f"  Content preview: {str(row.body_blob)[:100]}...")
        if hasattr(row, 'vector') and row.vector:
            print(f"  Vector dimensions: {len(row.vector)}")
        if hasattr(row, 'metadata_s'):
            print(f"  Metadata: {row.metadata_s}")

finally:
    # Restore the original pandas factory
    session.row_factory = original_row_factory
    print("\n=== Row factory restored ===")

In [ ]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
session.default_fetch_size = None

In [ ]:
rows = session.execute('SELECT * from squad')
    
df_baseline = rows._current_rows
df_baseline

In [ ]:
column_types = df_baseline.dtypes

print(column_types)

In [ ]:
df_baseline['answers'] = df_baseline['answers'].apply(lambda x : str(x))

In [ ]:
pip freeze 